In [1]:
import pickle
import sys
from math import sqrt, pi
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import det, inv
import seaborn as sns
%matplotlib inline

In [2]:
X_train, y_train, X_test, y_test = pickle.load(open('data/sample.pickle','rb'),encoding = 'latin1')

In [3]:
X_train.shape, y_train.shape

((150, 2), (150, 1))

In [4]:
X_test.shape, y_test.shape

((100, 2), (100, 1))

In [161]:
def feature_mean(samples, labels):
    distinct_labels = np.unique(labels)
    means = []
    
    for label in distinct_labels:
        means.append(samples[np.flatnonzero(labels == label)].mean(0))
        
    return np.asarray(means).T

In [185]:
def feature_cov(samples, labels):
    distinct_labels = np.unique(labels)
    covariances = []
    
    for label in distinct_labels:
        covariances.append(np.cov(samples[np.flatnonzero(labels == label)].T))
        
    return np.asarray(covariances)

In [7]:
def ldaLearn(X,y):    
    means = feature_mean(X, y)
    covmat = np.cov(X.T)
    return means, covmat

In [83]:
def ldaTest(means, covmat, Xtest, ytest):
    preds = np.argmax(
        np.apply_along_axis(pdf, 0, means, covmat, Xtest),
        axis=1,
    ) + 1
    acc = np.mean(ytest.reshape((Xtest.shape[0],)) == preds)
    preds = preds.reshape((Xtest.shape[0], 1))
    return acc, preds

In [195]:
def pdf(mean, sigma, Xtest):
    K = 1/sqrt(((2 * pi) ** len(mean)) * det(sigma))
    return K * np.exp(-0.5 * ((Xtest - mean) @ inv(sigma) * (Xtest - mean)).sum(axis=1))

In [202]:
# LDA
means,covmat = ldaLearn(X_train, y_train)
ldaacc,ldares = ldaTest(means,covmat,X_test,y_test)
print('LDA Accuracy = '+str(ldaacc))

LDA Accuracy = 0.97


In [139]:
def qdaLearn(X,y):
    means = feature_mean(X, y)
    covmats = feature_cov(X, y)
    return means, covmats

In [224]:
def qdaTest(means, covmats, Xtest, ytest):
    preds = np.asarray(
        [pdf(means.T[i], covmats[i], Xtest) for i in range(means.shape[1])],
    ).T
    preds = np.argmax(preds, axis=1) + 1
    acc = np.mean(ytest.reshape((Xtest.shape[0],)) == preds)
    preds = preds.reshape((Xtest.shape[0], 1))

    return acc, preds

In [230]:
means,covmats = qdaLearn(X_train, y_train)
print(means.shape, covmats.shape)
qdaacc,qdares = qdaTest(means, covmats, X_test, y_test)
print('QDA Accuracy = '+str(qdaacc))

(2, 5) (5, 2, 2)
QDA Accuracy = 0.96
